In [2]:
%pip install pyspark

  Using cached pyspark-4.0.0.tar.gz (434.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
  Running setup.py install for pyspark: started
  Running setup.py install for pyspark: still running...
  Running setup.py install for pyspark: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from pyspark.sql import SparkSession

import os
#os.environ["SPARK_LOCAL_DIR"] = "C:/tmp/spark"
os.environ["SPARK_LOCAL_DIR"] = "C:/Users/thais/spark-temp"

In [2]:
spark = SparkSession.builder.master("local").appName("Test").getOrCreate()
df = spark.createDataFrame([(1, "Hello"), (2, "World")], ["id", "message"])
df.show()

+---+-------+
| id|message|
+---+-------+
|  1|  Hello|
|  2|  World|
+---+-------+



In [13]:
%pip install esgf-pyclient==0.3.1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
     -------------------------------------- 134.9/134.9 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 64.8/64.8 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 61.4/61.4 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 115.4/115.4 kB 6.6 MB/s eta 0:00:00
     ---------------------------------------- 45.4/45.4 kB ? eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 105.4/105.4 kB ? eta 0:00:00
  Using cached idna-3.10-py3-none-any.whl (70 kB)
     -------------------------------------- 129.8/129.8 kB 8.0 MB/s eta 0:00:00
     ------------------------------------- 162.7/162.7 kB 10.2 MB/s eta 0:00:00
  Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
     ---------------------

  DEPRECATION: esgf-pyclient is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### listagem de variáveis do CMPI6

In [1]:
import sys

# apontar para a versão local da lib
sys.path.insert(0, './dreqPy')

In [2]:
from dreqPy import dreq

In [3]:
# inicializando a biblioteca
dq = dreq.loadDreq()

In [4]:
# listando variáveis relacionadas à temperatura
temperature_vars = [var for var in dq.coll["var"].items if "temperature" in var.title.lower()]

for v in temperature_vars:
    print(v.label, "-", v.title, "-", v.sn)

bigthetao - Sea Water Conservative Temperature - sea_water_conservative_temperature
bigthetaoga - Global Average Sea Water Conservative Temperature - sea_water_conservative_temperature
edt - Eddy Diffusivity Coefficient for Temperature - atmosphere_heat_diffusivity
hfevapds - Temperature Flux Due to Evaporation Expressed as Heat Flux out of Sea Water - temperature_flux_due_to_evaporation_expressed_as_heat_flux_out_of_sea_water
hfrainds - Temperature Flux Due to Rainfall Expressed as Heat Flux into Sea Water - temperature_flux_due_to_rainfall_expressed_as_heat_flux_into_sea_water
hfrunoffds - Temperature Flux Due to Runoff Expressed as Heat Flux into Sea Water - temperature_flux_due_to_runoff_expressed_as_heat_flux_into_sea_water
litempbotfl - Basal Temperature of Floating Ice Shelf - land_ice_basal_temperature
litempbotgr - Basal Temperature of Grounded Ice Sheet - land_ice_basal_temperature
litemptop - Temperature at Top of Ice Sheet Model - temperature_at_top_of_ice_sheet_model
litem

### acesso ao ESGF

In [3]:
from collections import Counter, defaultdict
import logging
from pyesgf.search import SearchConnection

In [4]:
logging.getLogger('pyesgf.search').setLevel(logging.ERROR)

In [5]:
# variáveis relevantes de acordo com GCOS
variaveis = ["tas", "tasmax", "tasmin", "ps", "pr", "hur", "rsds", "rlut", "uas", "vas", "tos", "sos", "sic"]

In [55]:
conn = SearchConnection("https://esgf-node.llnl.gov/esg-search", distrib=False)   # distrib = False —> força a busca apenas no LLNL, que costuma estar mais estável

ctx = conn.new_context(
    project="CMIP6",
    experiment_id=["historical", "ssp245", "ssp585"],
    variable_id=variaveis,
    frequency=["mon"],
    #member_id="r1i1p1f1",
    #grid_label="gn",
    replica=False,
    latest=True
)

# armazenando os resultados
results = list(ctx.search(ignore_facet_check=True))

# extraindo os modelos (source_id)
models = [d.json.get("source_id", [None])[0] for d in results if "source_id" in d.json]

# frequência de cada modelo
print(Counter(models))

"""
mapping = {}

for d in ctx.search(ignore_facet_check=True):
    models = [d.json.get("source_id", [None])[0] for d in ctx.search(ignore_facet_check=True)]
    print(Counter(models))

    model = d.json.get("source_id", [None])[0]
    experiment = d.json.get("experiment_id", [None])[0]
    variable = d.json.get("variable_id", [None])[0]
    
    if model and experiment and variable:
        mapping.setdefault(model, {}).setdefault(experiment, set()).add(variable)

# Exibir resultados
for model, exps in mapping.items():
    print(f"\nModelo: {model}")
    for exp, vars in exps.items():
        print(f"  Experimento: {exp} → Variáveis: {sorted(vars)}")"""

Counter()


'\nmapping = {}\n\nfor d in ctx.search(ignore_facet_check=True):\n    models = [d.json.get("source_id", [None])[0] for d in ctx.search(ignore_facet_check=True)]\n    print(Counter(models))\n\n    model = d.json.get("source_id", [None])[0]\n    experiment = d.json.get("experiment_id", [None])[0]\n    variable = d.json.get("variable_id", [None])[0]\n\n    if model and experiment and variable:\n        mapping.setdefault(model, {}).setdefault(experiment, set()).add(variable)\n\n# Exibir resultados\nfor model, exps in mapping.items():\n    print(f"\nModelo: {model}")\n    for exp, vars in exps.items():\n        print(f"  Experimento: {exp} → Variáveis: {sorted(vars)}")'

In [13]:
variaveis = ["tas", "tasmin", "tasmax", "tos", "ps", "pr"]  # exemplo com 6 variáveis
#variaveis = ["tas"]
#models = ["EC-Earth3", "CanESM5", "MPI-ESM1-2-LR", "IPSL-CM5A2-INCA"]
modelos_por_variavel = defaultdict(set)

conn = SearchConnection("https://esgf-node.llnl.gov/esg-search", distrib=False, timeout=300)
#conn = SearchConnection("https://esgf-node.ornl.gov/esg-search", distrib=False)
#conn = SearchConnection("https://eagle.alcf.anl.gov/esg-search", distrib=False)

for var in variaveis:
    ctx = conn.new_context(
        project="CMIP6",
        experiment_id=["historical", "ssp245", "ssp585"],
        variable_id=var,
        frequency="mon",
        #source_id="MIROC6",
        replica=False,
        latest=True
    )

    results = list(ctx.search(ignore_facet_check=True))
    for d in results:
        modelo = d.json.get("source_id", [None])[0]
        if modelo:
            modelos_por_variavel[var].add(modelo)

# Exibindo os resultados
for var, modelos in modelos_por_variavel.items():
    print(f"{var}: {sorted(modelos)}")

tas: ['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CIESM', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'CanESM5', 'CanESM5-1', 'CanESM5-CanOE', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA', 'EC-Earth3', 'EC-Earth3-CC', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'FGOALS-f3-L', 'FGOALS-g3', 'FIO-ESM-2-0', 'GFDL-ESM4', 'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'GISS-E2-2-G', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'IITM-ESM', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'KACE-1-0-G', 'KIOST-ESM', 'MCM-UA-1-0', 'MIROC-ES2H', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'TaiESM1', 'UKESM1-0-LL']
tasmin: ['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'CAS-ESM2-0', 'CESM2', 'CESM2-WACCM', 'CIESM', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'CanESM5', 'CanESM5-1', 'CanESM5-CanOE'

In [ ]:
# obtendo informações dos datasets para as variáveis

variaveis = ["tas", "tasmin", "tasmax", "tos", "ps", "pr"]  # exemplo com 6 variáveis
#variaveis = ["tas"]
models = "EC-Earth3" # ["EC-Earth3", "CanESM5", "MPI-ESM1-2-LR", "IPSL-CM5A2-INCA"]
modelos_por_variavel = defaultdict(set)
infos_datasets = {}

conn = SearchConnection("https://esgf-node.llnl.gov/esg-search", distrib=False, timeout=300)
#conn = SearchConnection("https://esgf-node.ornl.gov/esg-search", distrib=False)
#conn = SearchConnection("https://eagle.alcf.anl.gov/esg-search", distrib=False)

for var in variaveis:
    ctx = conn.new_context(
        project="CMIP6",
        experiment_id=["historical", "ssp245", "ssp585"],
        variable_id=var,
        frequency="mon",
        source_id=models,
        replica=False,
        latest=True
    )

    results = list(ctx.search(ignore_facet_check=True))
    infos_datasets[var] = results

    for d in results:
        modelo = d.json.get("source_id", [None])[0]
        
        if modelo:
            modelos_por_variavel[var].add(modelo)

# Exibindo os resultados
for var, modelos in modelos_por_variavel.items():
    print(f"{var}: {sorted(modelos)}")

tas: ['EC-Earth3']
tasmin: ['EC-Earth3']
tasmax: ['EC-Earth3']
tos: ['EC-Earth3']
ps: ['EC-Earth3']
pr: ['EC-Earth3']


In [36]:
infos_datasets["tas"]

In [39]:
agrupamento = defaultdict(set)

for var in variaveis:
    print(f"quantidadede de DatasetResults da variável {var}: {len(infos_datasets[var])}")

    for d in infos_datasets["tas"]:
        experimento = d.json.get("experiment_id", ["-"])[0]
        membro = d.json.get("member_id", ["-"])[0]
        agrupamento[experimento].add(membro)

    for exp, membros in agrupamento.items():
        print(f"{exp}: {len(membros)} membros\n-----\n")

quantidadede de DatasetResults da variável tas: 178
ssp585: 58 membros
-----

quantidadede de DatasetResults da variável tasmin: 178
ssp585: 58 membros
-----

quantidadede de DatasetResults da variável tasmax: 178
ssp585: 58 membros
-----

quantidadede de DatasetResults da variável tos: 171
ssp585: 58 membros
-----

quantidadede de DatasetResults da variável ps: 53
ssp585: 58 membros
-----

quantidadede de DatasetResults da variável pr: 178
ssp585: 58 membros
-----



In [ ]:
# identificando os datasets mais recentes de cada variável
from datetime import datetime

dataset_mais_recente_por_variavel = {}

for var in variaveis:
    datasets = infos_datasets[var]

    # Filtrar apenas datasets com creation_date válido
    datasets_validos = [
        d for d in datasets if d.json.get("creation_date") is not None
    ]

    if not datasets_validos:
        print(f"[{var}] Nenhum dataset com creation_date disponível.")
        continue

    # Selecionar o dataset mais recente
    dataset_mais_recente = max(
        datasets_validos,
        key=lambda d: datetime.fromisoformat(d.json["creation_date"].replace("Z", ""))
    )

    dataset_mais_recente_por_variavel[var] = dataset_mais_recente

    print(f"[{var}] Dataset mais recente:")
    print(f"  ID.............: {dataset_mais_recente.dataset_id}")
    print(f"  Experimento....: {dataset_mais_recente.json.get('experiment_id', ['?'])[0]}")
    print(f"  Membro.........: {dataset_mais_recente.json.get('member_id', ['?'])[0]}")
    print(f"  Data criação...: {dataset_mais_recente.json.get('creation_date')}")
    print(f"  Número arquivos: {dataset_mais_recente.json.get('number_of_files')}")
    print("")

[tas] Dataset mais recente:
  ID.............: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r1i1p1f1.Amon.tas.gr.v20200310|eagle.alcf.anl.gov
  Experimento....: ssp585
  Membro.........: r1i1p1f1
  Data criação...: 2019-09-27T17:17:08Z
  Número arquivos: 86

[tasmin] Dataset mais recente:
  ID.............: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r1i1p1f1.Amon.tasmin.gr.v20200310|eagle.alcf.anl.gov
  Experimento....: ssp585
  Membro.........: r1i1p1f1
  Data criação...: 2019-09-27T20:42:13Z
  Número arquivos: 86

[tasmax] Dataset mais recente:
  ID.............: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r1i1p1f1.Amon.tasmax.gr.v20200310|eagle.alcf.anl.gov
  Experimento....: ssp585
  Membro.........: r1i1p1f1
  Data criação...: 2019-09-27T19:55:35Z
  Número arquivos: 86

[tos] Dataset mais recente:
  ID.............: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r1i1p1f1.Omon.tos.gn.v20200918|esgf-node.ornl.gov
  Experimento....: ssp585
  M

In [43]:
download_dir = r"C:\Users\thais\OneDrive\Documentos\#POLI\2024_4\Trabalho de Conclusão de Curso\cmip6_ingestion\datasets\ec_earth3"  # ajuste para o diretório desejado
os.makedirs(download_dir, exist_ok=True)

for var, dataset in dataset_mais_recente_por_variavel.items():
    print(f"\n[{var}] Localizando arquivos...")
    files = list(dataset.file_context().search())
    
    print(f"  Total de arquivos encontrados: {len(files)}")

    # Apenas arquivos HTTP disponíveis
    files_http = [f for f in files if "HTTPServer" in f.json.get("access", [])]

    if not files_http:
        print(f"  Nenhum arquivo com acesso HTTP disponível para {var}")
        continue

    for f in files_http:
        url = f.download_url()
        filename = os.path.join(download_dir, var, os.path.basename(url))
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        # Comando para baixar com wget
        cmd = f"wget -nc -O {filename} {url}"
        print(f"  Baixando: {url}")
        os.system(cmd)  # ou subprocess.run([...]) se preferir controle fino



[tas] Localizando arquivos...


HTTPError: 422 Client Error: Unprocessable Content for url: https://esgf-node.ornl.gov/esgf-1-5-bridge?format=application%2Fsolr%2Bjson&limit=0&distrib=false&type=File&dataset_id=CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r1i1p1f1.Amon.tas.gr.v20200310%7Ceagle.alcf.anl.gov&facets=%2A

In [45]:
# Conexão apenas para arquivos
#conn_files = SearchConnection("https://esgf-data.dkrz.de/esg-search", distrib=False)
conn_files = SearchConnection("https://esgf-node.llnl.gov/esg-search", distrib=False, timeout=300)

for var, dataset in dataset_mais_recente_por_variavel.items():
    dataset_id = dataset.dataset_id
    print(f"\n[{var}] Buscando arquivos para: {dataset_id}")

    # Criar contexto no nó de download com o mesmo dataset_id
    ctx = conn_files.new_context(project="CMIP6", dataset_id=dataset_id, type="File")

    try:
        files = list(ctx.search())
    except Exception as e:
        print(f"  Erro ao buscar arquivos: {e}")
        continue

    print(f"  Total de arquivos encontrados: {len(files)}")

    files_http = [f for f in files if "HTTPServer" in f.json.get("access", [])]

    if not files_http:
        print(f"  Nenhum arquivo com acesso HTTP disponível para {var}")
        continue

    for f in files_http:
        url = f.download_url()
        filename = os.path.join(download_dir, var, os.path.basename(url))
        os.makedirs(os.path.dirname(filename), exist_ok=True)

        cmd = f"wget -nc -O \"{filename}\" {url}"
        print(f"  Baixando: {url}")
        os.system(cmd)



[tas] Buscando arquivos para: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r1i1p1f1.Amon.tas.gr.v20200310|eagle.alcf.anl.gov
  Erro ao buscar arquivos: 422 Client Error: Unprocessable Content for url: https://esgf-node.ornl.gov/esgf-1-5-bridge?format=application%2Fsolr%2Bjson&limit=0&distrib=false&type=Dataset&project=CMIP6&dataset_id=CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r1i1p1f1.Amon.tas.gr.v20200310%7Ceagle.alcf.anl.gov&facets=%2A

[tasmin] Buscando arquivos para: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r1i1p1f1.Amon.tasmin.gr.v20200310|eagle.alcf.anl.gov
  Erro ao buscar arquivos: 422 Client Error: Unprocessable Content for url: https://esgf-node.ornl.gov/esgf-1-5-bridge?format=application%2Fsolr%2Bjson&limit=0&distrib=false&type=Dataset&project=CMIP6&dataset_id=CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r1i1p1f1.Amon.tasmin.gr.v20200310%7Ceagle.alcf.anl.gov&facets=%2A

[tasmax] Buscando arquivos para: CMIP6.ScenarioMIP.EC-Eart

In [27]:
infos_datasets["tas"][0].json

{'mip_era': ['CMIP6'],
 'activity_drs': ['ScenarioMIP'],
 'institution_id': ['EC-Earth-Consortium'],
 'source_id': ['EC-Earth3'],
 'experiment_id': ['ssp585'],
 'member_id': ['r131i1p1f1'],
 'table_id': ['Amon'],
 'variable_id': ['tas'],
 'grid_label': ['gr'],
 'frequency': ['mon'],
 'realm': ['atmos'],
 'product': ['model-output'],
 'nominal_resolution': ['100 km'],
 'source_type': ['AOGCM'],
 'grid': ['T255L91-ORCA1L75'],
 'creation_date': '2019-08-19T11:49:59Z',
 'variant_label': ['r131i1p1f1'],
 'sub_experiment_id': ['none'],
 'further_info_url': ['https://furtherinfo.es-doc.org/CMIP6.EC-Earth-Consortium.EC-Earth3.ssp585.none.r131i1p1f1'],
 'activity_id': ['ScenarioMIP'],
 'data_specs_version': ['01.00.30'],
 'title': 'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r131i1p1f1.Amon.tas.gr',
 'experiment_title': ['update of RCP8.5 based on SSP5'],
 'data_node': 'eagle.alcf.anl.gov',
 'index_node': 'us-index',
 'master_id': 'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp5

In [21]:
infos_datasets

{'tas': <pyesgf.search.results.DatasetResult at 0x15bb28d6790>,
 'tasmin': <pyesgf.search.results.DatasetResult at 0x15bb257a7d0>,
 'tasmax': <pyesgf.search.results.DatasetResult at 0x15bb26ea850>,
 'tos': <pyesgf.search.results.DatasetResult at 0x15bb2855f10>,
 'ps': <pyesgf.search.results.DatasetResult at 0x15bb287db90>,
 'pr': <pyesgf.search.results.DatasetResult at 0x15bb285b5d0>}

In [16]:
results[0].json

{'mip_era': ['CMIP6'],
 'activity_drs': ['ScenarioMIP'],
 'institution_id': ['EC-Earth-Consortium'],
 'source_id': ['EC-Earth3'],
 'experiment_id': ['ssp585'],
 'member_id': ['r4i1p1f1'],
 'table_id': ['Amon'],
 'variable_id': ['pr'],
 'grid_label': ['gr'],
 'frequency': ['mon'],
 'realm': ['atmos'],
 'product': ['model-output'],
 'nominal_resolution': ['100 km'],
 'source_type': ['AOGCM'],
 'grid': ['T255L91'],
 'creation_date': '2019-07-29T11:46:35Z',
 'variant_label': ['r4i1p1f1'],
 'sub_experiment_id': ['none'],
 'further_info_url': ['https://furtherinfo.es-doc.org/CMIP6.EC-Earth-Consortium.EC-Earth3.ssp585.none.r4i1p1f1'],
 'activity_id': ['ScenarioMIP'],
 'data_specs_version': ['01.00.30'],
 'title': 'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r4i1p1f1.Amon.pr.gr',
 'experiment_title': ['update of RCP8.5 based on SSP5'],
 'data_node': 'eagle.alcf.anl.gov',
 'index_node': 'us-index',
 'master_id': 'CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r4i1p1f1.Amon.pr

In [ ]:
d.json["source_id"]

{'mip_era': ['CMIP6'],
 'activity_drs': ['ScenarioMIP'],
 'institution_id': ['MIROC'],
 'source_id': ['MIROC6'],
 'experiment_id': ['ssp585'],
 'member_id': ['r7i1p1f1'],
 'table_id': ['Amon'],
 'variable_id': ['tas'],
 'grid_label': ['gn'],
 'frequency': ['mon'],
 'realm': ['atmos'],
 'product': ['model-output'],
 'nominal_resolution': ['250 km'],
 'source_type': ['AOGCM', 'AER'],
 'grid': ['native atmosphere T85 Gaussian grid'],
 'creation_date': '2020-06-13T21:31:53Z',
 'variant_label': ['r7i1p1f1'],
 'sub_experiment_id': ['none'],
 'further_info_url': ['https://furtherinfo.es-doc.org/CMIP6.MIROC.MIROC6.ssp585.none.r7i1p1f1'],
 'activity_id': ['ScenarioMIP'],
 'data_specs_version': ['01.00.31'],
 'title': 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp585.r7i1p1f1.Amon.tas.gn',
 'experiment_title': ['update of RCP8.5 based on SSP5'],
 'model_cohort': ['Registered'],
 'data_node': 'esgf-node.ornl.gov',
 'index_node': 'us-index',
 'master_id': 'CMIP6.ScenarioMIP.MIROC.MIROC6.ssp585.r7i1p1f1.Amon.